In [1]:
import gzip
import pandas as pd
import numpy as np
import nltk
import requests
from io import BytesIO
import imblearn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet') 
import re
import contractions
from sklearn.preprocessing import LabelEncoder
from bs4 import BeautifulSoup 
pd.set_option('display.max_colwidth', None) 
import requests
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import gensim.downloader as api
from gensim.models import Word2Vec
wv = api.load('word2vec-google-news-300')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/khalid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/khalid/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/khalid/opt/anaconda3/envs/EE660/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Functions

In [2]:
def read_file(link):
    url = link
    data_compressed = requests.get(url).content
    data = pd.read_csv(BytesIO(data_compressed), compression='gzip', sep='\t',error_bad_lines=False, warn_bad_lines=False)
    return data


# Remove HTML tags funtion
def remove_tags(txt):
    # parse html content
    soup = BeautifulSoup(txt, "html.parser")
    
    # get tags content
    for data in soup(['style', 'script']): 
        data.get_text()
        
    # return html's tag content
    return ' '.join(soup.stripped_strings)

# Remove URLS funtion
def remove_urls(txt):
    return re.sub(r"http\S+", "", txt)

# Apply contraction to words
def contractionfunction(s):
    expanded_words = [] 
    for word in s.split():
        expanded_words.append(contractions.fix(word))
    result = ' '.join(expanded_words)
    return result

def remove_non_alphabetical(txt):
    regex = re.compile('[\W_0-9]+')
    dirty_list = txt.split()
    clean_list = [regex.sub(' ', word) for word in dirty_list] 
    clean_string = ' '.join(clean_list)
    return clean_string



# remove stop words function
def remove_stop_words(txt):
    
    stop = stopwords.words('english') 
    word_list = txt.split() 
    clean_list = []
    clean_string = ''
    for word in word_list:
        if word not in stop: 
            clean_list.append(word)
    clean_string = ' '.join(clean_list) 
    return clean_string

def leammatize_review(txt): 
    lemmatizer = WordNetLemmatizer() 
    word_list = txt.split() 
    clean_list = []
    clean_string = ''
    for word in word_list:
        new_word = lemmatizer.lemmatize(word)
        clean_list.append(new_word) 
    clean_string = ' '.join(clean_list) 
    return clean_string

def review_num_words(txt):
    return len(txt.split())

# Data

In [3]:
data_kitchen = read_file("https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz")
#data_home = read_file("https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz")

/var/folders/qf/k0f0k7s94bd63pjpts_176c40000gn/T/ipykernel_25230/1245615390.py:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  data_kitchen = read_file("https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz")
/var/folders/qf/k0f0k7s94bd63pjpts_176c40000gn/T/ipykernel_25230/1245615390.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  data_kitchen = read_file("https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz")


# Pre Processing

In [4]:
###### Pre Processing 

#removed the na values
data_kitchen = data_kitchen.dropna().reset_index(drop=True)

#removed the 3 dates from the product category
a = data_kitchen['product_category'] == 'Kitchen'
data_kitchen = data_kitchen[a]

#make decoder
data_kitchen['vine'] = data_kitchen['vine'].astype('category').cat.codes
data_kitchen['verified_purchase'] = data_kitchen['verified_purchase'].astype('category').cat.codes

#removing unnecessary columns 
data_kitchen = data_kitchen.drop( columns=['customer_id', 'product_id', 'marketplace', 'product_category', 'review_id', 'product_parent', 'product_title'] ) 

#combining two features in one feature 
data_kitchen['reviews'] = data_kitchen['review_headline'] + ' ' + data_kitchen['review_body']
data_kitchen = data_kitchen.drop( columns=['review_headline', 'review_body'] )  # drop the 2 columns
data_kitchen['reviews'] = data_kitchen['reviews'].str.lower()  # make lowercase

# make time
data_kitchen["review_date"] = pd.to_datetime(data_kitchen["review_date"])
data_kitchen["review_date"] = data_kitchen["review_date"].dt.isocalendar().week

print(data_kitchen.shape)
data_kitchen.head(10)

(4874562, 7)


,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_date,reviews
0,5.0,0.0,0.0,0,1,36,beautiful. looks great on counter beautiful. looks great on counter.
1,5.0,0.0,1.0,0,1,36,"awesome & self-ness i personally have 5 days sets and have also bought 2 sets for other people in my home. the two other sets i have decided to keep for myself. the purpose of keeping them for myself is to use them for other other than salt and pepper. they stay perfect, i use them constantly! i have a couple of people here that use them, say that there's just awesome. i did have a salt shaker that had a little problem converting from sea salt to himalaya salt. did not fall out correctly so what i did was i just took a top of part which is really simple it's like five pieces total and just kind of cleaned around the teflon and you know that the salt buildup from there was caused by humidity, cleaned out, my gosh it's unbelievable how much better, its almost better than new thank you bavaria these are top of the line in my book i hope in the near future you have some more come out. i sure would like to buy some more and some for my kids and family for christmas otherwise i'm keeping what i have. i think i deserve it. thank you.... don't know if this makes much sense doesn't need to, self-ness !!!"
2,5.0,0.0,0.0,0,1,36,fabulous and worth every penny fabulous and worth every penny. used for cleaning corn from the cob in seconds :) would recommend its purchase
3,5.0,0.0,1.0,0,1,36,five stars a must if you love garlic on tomato marinara sauce.
4,5.0,0.0,0.0,0,1,36,better than sex worth every penny! buy one now and be a pizza slice master!!
5,1.0,1.0,1.0,0,1,36,does not work on induction stoves! (not suitable for all type of surfaces) the description says &#34;suitable for all type of surfaces&#34; but it is not true!<br /><br />we ordered this item for our large family (we like to cook a lot) but this does not work as advertised!<br /><br />this pressure cooker does not work for induction stoves. i called the manufacturer (magefesa) and they confirmed that it will not work. we are exchanging this pot for the stainless steel version that is supposed to work on induction stoves. magefesa said that the 14.3 quart pot will work on induction stoves and the description here on amazon clearly states that it will work on induction stoves.<br /><br />thankfully amazon has a great return process so they will send a truck to pick this up and they will send the other version in a couple days.
6,5.0,0.0,0.0,0,1,36,awesome! first fryer i have owned and what a ... awesome! first fryer i have owned and what a luxury item! love the magnetic cord attachment. the baskets are a great size for home use and gets up to temp no problem. couldn't be happier with my purchase.
7,5.0,0.0,0.0,0,1,36,five stars very good item. quick delivery.
8,5.0,0.0,0.0,0,1,36,five stars sharp and look great
9,3.0,0.0,0.0,0,1,36,three stars should have come with a kit to install drain tube system.


In [27]:
data_kitchen['star_rating'].value_counts()

1    3856246
0    1018316
Name: star_rating, dtype: int64

In [28]:
data_kitchen['star_rating'] = np.where(data_kitchen['star_rating'] > 3.0 , 1, 0)

In [29]:
data_kitchen['star_rating'].value_counts()

0    4874562
Name: star_rating, dtype: int64

In [25]:
data_k_sampled = data_kitchen.sample(n=700000, replace=False, ignore_index = True)

In [26]:
data_k_sampled['star_rating'].value_counts()

1    553431
0    146569
Name: star_rating, dtype: int64

In [30]:
data_k_sampled

,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_date,reviews
0,0,5.0,5.0,0,1,21,"one star cheap, could only use these for craft supplies. even that, they break and split."
1,0,1.0,1.0,0,1,52,not worth your time. i can do better and faster. not even work the time to return. trash bin it went.
2,0,1.0,1.0,0,1,26,one star this microwave died after 3 months of normal use. the manufacturer did nothing to help.
3,1,1.0,2.0,0,0,8,"what you should know before you buy! i was provided a freddie and sebbie freezable lunch bag/cooler bag free for testing and review and was asked only to give my honest review, so this is what i found.<br /><br />what you should know before you buy:<br />♦ this is a medium sized lunch bag/cooler bag. large enough to hold quite a few items.<br />♦ this lunch bag has the reusable ice packs built right into the sides of the bag, so no fumbling around with ice packs that take up space in your typical bag.<br />♦ very nice quality and seems to be quite sturdy.<br />♦ did a great job of keeping my lunch cold at work.<br />♦ folds up pretty small with a velcro closure so it is easy to store in your freezer while you are waiting for it to cool off.<br />♦ has extra pockets on the sides.<br /><br />conclusion:<br />overall, this is great cooler bag/lunch bag. this bag has the reusable ice packs that you normally put in the bag built right into the walls of the bag. this is great because you don't have to take up bag space with separate ice packs or have to fumble through them to get to your food. it has a really nice, compact design when folded, so it doesn't take up much space while it is sitting in your freezer cooling off. when opened up, it holds quite a bit of food and drink items. i take my lunch to work pretty much every day, so i gave this bag a try and it did a great job keeping my lunch perfectly cold. we always take a small cooler or cooler bag with us on day trips that have lunch items and/or drinks for our family of four, and this bag can easily accommodate all of that. if you are in the market for a lunch bag and the integrated ice packs sound like a feature you would like, this bag is a great choice."
4,0,19.0,26.0,0,0,27,"difficult to suck i was very excited to order these popsicle molds after reading all the positive reviews. they are very easy to make and look lovely. eating them is another story! a popsicle should fit comfortably in any size mouth and i'm a woman with a very average size mouth. the popsicle was too big and has too many ridges to allow you to suck it...you never have enough surface area in your mouth at one time to melt any of the juice! it's not what you expect from a popsicle at all! i now know that a popsicle needs smooth sides, not ridges...it doesn't work for me at all and it drips!"
...,...,...,...,...,...,...,...
699995,1,0.0,0.0,0,1,24,best 14-inch pan i've found perfect for large portion cooking. very sturdy and the non stick surface works as well as any pan i have used.<br /><br />hand wash for maximum life of the non stick surface -- there's something in most dish washer detergents that is very hard on any non stick surface. the surface on this pan is so slick that a couple of swipes with a soapy dish cloth will take anything right off.<br /><br />highly recommended.
699996,0,0.0,0.0,0,1,33,"the pitcher remains cloudy only after 3 uses. the ... the pitcher remains cloudy only after 3 uses. the spout opening is flimsy and will not stay locked on. i would have given it a 2 star, but the fruit infused water does meet my expectation."
699997,1,0.0,0.0,0,1,25,awesome i love this magnet. it is high quality and looks awesome. i absolutely cannot complain. not to mention the shipping was super fast! thanks!
699998,1,0.0,0.0,0,1,31,"the perfect machine for the job i love this product. i bought this one as a gift but i've had mine for decades. it works perfectly and making readying apples for cooking super easy and super fast. i have searched for other apple p

In [31]:
# Applying preproccessing
data_k_sampled['reviews'] = data_k_sampled['reviews'].apply(lambda body :remove_tags(body))
data_k_sampled['reviews'] = data_k_sampled['reviews'].apply(lambda body :remove_urls(body))
data_k_sampled['reviews'] = data_k_sampled['reviews'].apply(lambda body :contractionfunction(body))
data_k_sampled['reviews'] = data_k_sampled['reviews'].apply(lambda body :remove_non_alphabetical(body))
data_k_sampled['reviews'] = data_k_sampled['reviews'].apply(lambda review:remove_stop_words(review))
data_k_sampled['reviews'] = data_k_sampled['reviews'].apply(lambda txt :leammatize_review(txt))

In [58]:
data_k = data_k_sampled.copy()

In [59]:
data_k['star_rating'].value_counts()

1    553431
0    146569
Name: star_rating, dtype: int64

In [60]:
from sklearn.utils import resample
df_majority = data_k[data_k["star_rating"]==1]
df_minority = data_k[data_k["star_rating"]==0]
df_majority_downsampled = resample(df_majority,
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority),     # to match minority class
                                 random_state=123) # reproducible results
# Combine minority class with downsampled majority class
data_k_concat = pd.concat([df_majority_downsampled, df_minority])
# Display new class counts
data_k_concat['star_rating'].value_counts()

1    146569
0    146569
Name: star_rating, dtype: int64

In [62]:
data_k_concat = data_k_concat.reset_index(drop=True)

In [63]:
data_k_concat

,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_date,reviews
0,1,51.0,51.0,0,1,9,work great novice candy maker bit scared use one thermometer worked great easy read made homemade marshallows exactly needed
1,1,0.0,0.0,0,1,41,never expected searching top flight pepper mill reading review ordered oxo good grip pepper mill product awesome volume flake size forehead sweating food hot enough fresh ground pepper help quest product fantastic say fantastic
2,1,0.0,0.0,0,1,8,wonderful work wonderful easy use came great recipe aebleskivers stick following instruction fall right making pefect pan ton fun using
3,1,0.0,0.0,0,1,34,great quality great color went piece space case lpve smooth heavy space case great quality great color I glad got steal price
4,1,0.0,0.0,0,1,28,useful replaced one used year expect one useful camping road trip etc
...,...,...,...,...,...,...,...
293133,0,0.0,0.0,0,1,28,three star ok
293134,0,15.0,16.0,0,0,28,buy cheap knockoff scam anyone ever tell seems good true probably well knew better bought anyway amazingly low price anyone know decent banana sliced least seeing lost head clicked buy button soon got knew cheap knockoff probably made china con warranty slicer gave one banana horrible customer service called demanded money back said would take least business day wtf bank sorry sign lend money amazon banana scam terrible color choice good luck finding banana piece mixed slicer accidentally ate piece slicer nearly died instruction idea start way turn banana peel first leave peel sorry gourmet chef instruction booklet would useful pro none
293135,0,3.0,4.0,0,1,36,work well dried sea salt seems like really good grinder sure easy hold use find grind well dried fleur de sel bought amazon maybe salt salt specifically designated grinder friendly sure problem hard recommend
293136,0,9.0,9.0,0,1,11,poor quality manufacture printing disappointed quality coaster expected much better coaster clearly white half slight yellow ish coloration lot yellow discoloration edge coaster excessive amount glazing element dripping edge dried porcelain yellow stain even colored packaging bit yellow addition coaster seem poor quality print spottiness I returning soon get chance really disappointing much wanted


In [64]:
y = data_k_concat['star_rating']
X = data_k_concat.drop(columns =['star_rating'])

In [65]:
indexs = X['reviews'].index
X['google_word2vec'] = pd.Series(dtype=object)
for idx , review in zip(indexs, X['reviews']):
    unseen_words = 0
    n = len(review.split())
    x = 0
    for word in review.split():
        try:
            x = x + wv[word]
        except KeyError:
            unseen_words = unseen_words + 1
    if unseen_words == n:
        X.at[idx, 'google_word2vec'] = np.NaN
        continue
    x = x/(n-unseen_words)
    x1 = x.reshape(-1, 1)
    x1 = x1.T
    X.at[idx, 'google_word2vec'] = x1[0]

In [66]:
X_data = X.copy()
X_data = X_data.drop( columns=['reviews'] )

In [67]:
X_data.isna().sum()

helpful_votes         0
total_votes           0
vine                  0
verified_purchase     0
review_date           0
google_word2vec      12
dtype: int64

In [68]:
index_na = X_data.loc[pd.isna(X_data["google_word2vec"]), :].index
X_data = X_data.drop(index_na)

In [69]:
X_data.shape

(293126, 6)

In [70]:
y_data = y.copy()
y_data = y_data.drop(index_na)
y_data.shape

(293126,)

In [71]:
X_data = X_data.reset_index(drop=True)
y_data = y_data.reset_index(drop=True)

In [72]:
X_data

,helpful_votes,total_votes,vine,verified_purchase,review_date,google_word2vec
0,51.0,51.0,0,1,9,"[0.043111164, -0.009412978, 0.017001681, 0.10070801, -0.01950582, 0.029783461, 0.09827338, -0.069442324, 0.00504303, 0.06669447, 0.06746928, -0.109176636, -0.029086642, -0.014607747, -0.08432346, 0.05312093, 0.06363254, 0.07739258, 0.012125651, -0.104522705, 0.020956835, 0.13087294, 0.008290608, 0.053798676, 0.021396212, -0.050035264, -0.06092665, 0.045321755, -0.024315728, -0.0092909075, -0.027358161, 0.018141005, -0.0039435495, -0.023379855, 0.0421346, 0.00018056233, 0.019263374, 0.01530626, 0.0846795, 0.083852135, 0.1111518, -0.08039686, 0.17212592, -0.024376763, -0.032324895, 0.027286105, -0.054139033, 0.05094401, -0.02123854, 0.013142903, -0.054829914, 0.053521052, -0.027808296, -0.071876526, -0.030619303, 0.044833712, 0.065159164, -0.118006386, 0.020070395, -0.025552537, -0.053578693, 0.06951226, -0.10331556, -0.051201716, 0.06762017, -0.031789143, -0.07822333, 0.0380622, -0.02637397, 0.045051575, 0.081448026, 0.059139676, 0.1043413, -0.05417209, -0.16880968, -0.010518392, 0.074028865, 0.07558356, 0.025814481, 0.06929694, 0.014274597, -0.039175246, 0.07344733, -0.006245931, -0.038087633, -0.08363512, -0.0368042, 0.1151869, 0.04592853, 0.03438992, -0.005420261, -0.011522081, -0.046963163, -0.036787245, -0.016893174, -0.10346137, 0.05199602, 0.030963473, 0.0020898182, -0.0257704, ...]"
1,0.0,0.0,0,1,41,"[-0.0069059483, 0.063821234, 0.041082945, 0.040674098, -0.014671775, 0.008881513, 0.053462982, -0.11723417, 0.057921465, 0.121474326, 0.013371187, -0.10051413, -0.059484147, 0.018082563, -0.10670382, 0.05928309, -0.007194519, 0.07959523, -0.027881399, -0.09387131, -0.04881915, 0.029970955, 0.06072998, -0.020480886, 0.006492839, -0.0646856, -0.050005745, 0.11693618, 0.050608914, 0.0098392265, -0.029147878, -0.017540427, 0.012160357, -0.03584918, -0.08498607, -0.005053352, 0.0067982394, -0.035407573, 0.048472684, 0.017780976, 0.053495154, -0.14274372, 0.13602582, -0.018969368, -0.07463163, -0.15564683, 0.0011201747, -0.021469565, 0.020450369, 0.040609024, -0.008593391, 0.018644445, -0.06489608, -0.006889792, 0.017166138, 0.005570356, -0.008422852, -0.074957564, 0.07561134, -0.07616649, -0.053016774, 0.055786133, -0.119641475, -0.05316925, 0.03584918, 0.04771513, -0.045914594, -0.041008864, 0.021627314, 0.029835533, 0.051286697, -0.034017228, 0.0496889, -0.016616821, -0.19045033, -0.051412247, 0.027770996, 0.024994794, 0.023182588, 0.0019567152, 0.042848475, -0.025708366, -0.0010941449, 0.035553876, 0.0033569336, -0.07500682, -0.11225891, 0.15803258, 0.034312528, 0.030070586, 0.015170827, 0.02107149, -0.035930857, -0.018253103, -0.058747236, -0.0650527, 5.6939966e-06, 0.0019102658, 0.021086749, -0.0039978027, ...]"
2,0.0,0.0,0,1,8,"[0.030339291, 0.0396849, 0.050813373, 0.12044164, 0.0077594956, -0.016965365, 0.09022281, -0.08133738, -0.026020251, 0.059557464, -0.011858087, -0.0810322, -0.027909128, 0.0009058902, -0.07804148, 0.059679534, 0.08434095, 0.067916065, -0.033545244, -0.12472373, 0.026411232, 0.1403941, 0.01977539, 0.041388262, 0.037064403, -0.030378694, -0.03434673, 0.089599386, -0.0042853104, -0.007537842, -0.04081164, -0.013536955, 0.061827008, 0.027093185, 0.016723633, -0.022829557, 0.013331363, -0.028416684, 0.07611601, 0.06340669, 0.13658383, -0.068166636, 0.17797852, -0.015535857, -0.018935354, -0.065342955, -0.03414114, 0.018040707, 0.0583464, 0.031204676, -0.037809674, 0.04871248, -0.005673057, 0.004722194, -0.023120679, 0.0025104724, 0.012403689, -0.13642642, 0.030334473, -0.09983344, -0.057932, 0.08075272, -0.07457211, -0.04153603, 0.05402254, 0.0048057153, -0.092079565, -0.031400982, -0.025403475, 0.06503457, 0.07299845, 0.008329692, 0.12034205, -0.013988294, -0.08702971, -0.033693817, 0.0049599097, 0.054284345, 0.017578928, 0.1096866, -0.014030055, -0.045917712, 0.039325915, -0.007512143, 0.035903126, -0.072553135, -0.09068379, 0.090405114, -0.004548725, 0.04323618, 0.040122584, 0.01965332, -0.04855186, -0.039

In [73]:
x_review = np.array(X_data['google_word2vec'].values.tolist())
df_temp = pd.DataFrame(x_review, columns=[f"wv_{i}" for i in range(300)])

In [74]:
df_temp

,wv_0,wv_1,wv_2,wv_3,wv_4,wv_5,wv_6,wv_7,wv_8,wv_9,...,wv_290,wv_291,wv_292,wv_293,wv_294,wv_295,wv_296,wv_297,wv_298,wv_299
0,0.043111,-0.009413,0.017002,0.100708,-0.019506,0.029783,0.098273,-0.069442,0.005043,0.066694,...,-0.103058,-0.003955,-0.114782,0.019799,-0.105607,-0.052619,0.042555,-0.049991,0.016832,-0.047917
1,-0.006906,0.063821,0.041083,0.040674,-0.014672,0.008882,0.053463,-0.117234,0.057921,0.121474,...,-0.026797,-0.039454,-0.047541,0.066873,0.034781,-0.067570,0.033986,-0.008218,-0.010667,0.011889
2,0.030339,0.039685,0.050813,0.120442,0.007759,-0.016965,0.090223,-0.081337,-0.026020,0.059557,...,-0.109985,0.056882,-0.095061,0.009425,-0.040884,-0.083371,0.089539,-0.062484,-0.013627,-0.006785
3,0.039315,0.149349,-0.036877,0.069978,0.026164,-0.064729,0.134397,-0.061477,0.056106,0.111659,...,-0.082159,0.017270,-0.173549,-0.027745,-0.014704,-0.023193,0.032708,-0.128273,0.020790,-0.065505
4,0.049276,0.069214,0.063080,0.093363,-0.100800,0.056458,0.071045,-0.105606,0.038317,0.081787,...,-0.070882,0.039983,-0.040914,-0.021336,-0.037570,-0.023885,-0.034219,-0.055471,0.044899,-0.019174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293121,0.071772,0.026530,0.044800,0.058309,0.039836,0.014486,0.013509,-0.113118,0.092936,0.013916,...,0.069407,0.125570,-0.047201,-0.085815,-0.112691,-0.085286,-0.128092,-0.057739,-0.144043,0.010396
293122,0.022969,0.015931,0.011098,0.101491,-0.050752,0.033033,0.059749,-0.057849,0.084044,0.096586,...,-0.033710,0.053456,-0.089812,0.052977,0.013515,-0.016510,-0.006385,-0.070121,0.031087,-0.006506
293123,0.015420,0.067592,-0.001157,0.133979,-0.058613,0.047345,0.036396,-0.030024,0.034133,0.053513,...,-0.057383,-0.024374,-0.073713,0.088860,-0.025497,-0.005793,0.028541,-0.020043,0.020144,-0.051395
293124,0.038206,0.095898,-0.007226,0.060536,-0.058429,0.052176,0.098441,-0.130769,0.050816,0.137001,...,-0.053699,-0.014183,-0.094532,0.022514,0.021254,-0.018901,0.049044,-0.019012,0.041439,-0.069279


In [75]:
x_data_final = pd.concat([X_data, df_temp], axis=1)
x_data_final = x_data_final.drop(columns=['google_word2vec'])

In [76]:
x_data_final

,helpful_votes,total_votes,vine,verified_purchase,review_date,wv_0,wv_1,wv_2,wv_3,wv_4,...,wv_290,wv_291,wv_292,wv_293,wv_294,wv_295,wv_296,wv_297,wv_298,wv_299
0,51.0,51.0,0,1,9,0.043111,-0.009413,0.017002,0.100708,-0.019506,...,-0.103058,-0.003955,-0.114782,0.019799,-0.105607,-0.052619,0.042555,-0.049991,0.016832,-0.047917
1,0.0,0.0,0,1,41,-0.006906,0.063821,0.041083,0.040674,-0.014672,...,-0.026797,-0.039454,-0.047541,0.066873,0.034781,-0.067570,0.033986,-0.008218,-0.010667,0.011889
2,0.0,0.0,0,1,8,0.030339,0.039685,0.050813,0.120442,0.007759,...,-0.109985,0.056882,-0.095061,0.009425,-0.040884,-0.083371,0.089539,-0.062484,-0.013627,-0.006785
3,0.0,0.0,0,1,34,0.039315,0.149349,-0.036877,0.069978,0.026164,...,-0.082159,0.017270,-0.173549,-0.027745,-0.014704,-0.023193,0.032708,-0.128273,0.020790,-0.065505
4,0.0,0.0,0,1,28,0.049276,0.069214,0.063080,0.093363,-0.100800,...,-0.070882,0.039983,-0.040914,-0.021336,-0.037570,-0.023885,-0.034219,-0.055471,0.044899,-0.019174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293121,0.0,0.0,0,1,28,0.071772,0.026530,0.044800,0.058309,0.039836,...,0.069407,0.125570,-0.047201,-0.085815,-0.112691,-0.085286,-0.128092,-0.057739,-0.144043,0.010396
293122,15.0,16.0,0,0,28,0.022969,0.015931,0.011098,0.101491,-0.050752,...,-0.033710,0.053456,-0.089812,0.052977,0.013515,-0.016510,-0.006385,-0.070121,0.031087,-0.006506
293123,3.0,4.0,0,1,36,0.015420,0.067592,-0.001157,0.133979,-0.058613,...,-0.057383,-0.024374,-0.073713,0.088860,-0.025497,-0.005793,0.028541,-0.020043,0.020144,-0.051395
293124,9.0,9.0,0,1,11,0.038206,0.095898,-0.007226,0.060536,-0.058429,...,-0.053699,-0.014183,-0.094532,0.022514,0.021254,-0.018901,0.049044,-0.019012,0.041439,-0.069279


In [77]:
y_data.shape

(293126,)

In [78]:
np.save('X_data_Kitch_finalized.npy', x_data_final ) #data_k_concat
np.save('X_Kitch_review_wv.npy', df_temp ) #data_k_concat
np.save('y_data_Kitch_finalized.npy', y_data ) #data_k_concat